# Анализ данных VRP задачи E-CUP 2025

In [21]:
import sqlite3
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from pathlib import Path
import json

print("Библиотеки загружены")

Библиотеки загружены


In [ ]:
print("Загружаем данные...")

with open("ml_ozon_logistic/ml_ozon_logistic_dataSetOrders.json", 'r') as f:
    orders_data = json.load(f)
orders_df = pl.DataFrame(orders_data['Orders'])
print(f"✅ Заказы загружены: {len(orders_df)} записей")

with open("ml_ozon_logistic/ml_ozon_logistic_dataSetCouriers.json", 'r') as f:
    couriers_data = json.load(f)
couriers_df = pl.DataFrame(couriers_data['Couriers'])
warehouse_info = couriers_data['Warehouse']
print(f"✅ Курьеры загружены: {len(couriers_df)} записей")
print(f"🏢 Склад: ID={warehouse_info['ID']}, координаты=({warehouse_info['Lat']:.4f}, {warehouse_info['Long']:.4f})")

Загружаем данные...
✅ Заказы загружены: 20160 записей
✅ Курьеры загружены: 280 записей
🏢 Склад: ID=1, координаты=(55.8552, 36.8018)


In [ ]:
print("=== АНАЛИЗ ЗАКАЗОВ ===")
print(f"📊 Общее количество заказов: {len(orders_df)}")
print(f"🏘️ Уникальных микрополигонов: {orders_df.select('MpId').n_unique()}")

mp_stats = orders_df.group_by('MpId').len().sort('len', descending=True)
print(f"\n📈 Статистики по микрополигонам:")
print(f"   • Медиана заказов на полигон: {mp_stats['len'].median():.1f}")
print(f"   • Среднее заказов на полигон: {mp_stats['len'].mean():.1f}")
print(f"   • Максимум заказов в полигоне: {mp_stats['len'].max()}")
print(f"   • Минимум заказов в полигоне: {mp_stats['len'].min()}")
print(f"   • Стандартное отклонение: {mp_stats['len'].std():.1f}")

print(f"\n🏆 Топ-10 полигонов по количеству заказов:")
print(mp_stats.head(10))

=== АНАЛИЗ ЗАКАЗОВ ===
📊 Общее количество заказов: 20160
🏘️ Уникальных микрополигонов: 1394

📈 Статистики по микрополигонам:
   • Медиана заказов на полигон: 15.0
   • Среднее заказов на полигон: 14.5
   • Максимум заказов в полигоне: 15
   • Минимум заказов в полигоне: 1
   • Стандартное отклонение: 1.6

🏆 Топ-10 полигонов по количеству заказов:
shape: (10, 2)
┌──────────────────┬─────┐
│ MpId             ┆ len │
│ ---              ┆ --- │
│ i64              ┆ u32 │
╞══════════════════╪═════╡
│ 7628             ┆ 15  │
│ 5127             ┆ 15  │
│ 1021000000011749 ┆ 15  │
│ 7613             ┆ 15  │
│ 5410             ┆ 15  │
│ 9456             ┆ 15  │
│ 9852             ┆ 15  │
│ 5514             ┆ 15  │
│ 9197             ┆ 15  │
│ 1021000000006387 ┆ 15  │
└──────────────────┴─────┘


In [ ]:
print("=== АНАЛИЗ КУРЬЕРОВ ===")
print(f"🚚 Общее количество курьеров: {len(couriers_df)}")

service_times = []
for row in couriers_df.iter_rows(named=True):
    for service in row['ServiceTimeInMps']:
        service_times.append({
            'courier_id': row['ID'],
            'mp_id': service['MpID'],
            'service_time': service['ServiceTime']
        })

service_df = pl.DataFrame(service_times)
print(f"\n⏱️ Статистики сервисных времен:")
print(f"   • Общее количество записей: {len(service_df)}")
print(f"   • Уникальных полигонов: {service_df.select('mp_id').n_unique()}")
print(f"   • Диапазон времени: {service_df['service_time'].min()} - {service_df['service_time'].max()} сек")
print(f"   • Среднее время: {service_df['service_time'].mean():.1f} сек")
print(f"   • Медиана времени: {service_df['service_time'].median():.1f} сек")

mp_per_courier = service_df.group_by('courier_id').len().sort('len', descending=True)
print(f"\n📊 Полигоны на курьера:")
print(f"   • Среднее: {mp_per_courier['len'].mean():.1f}")
print(f"   • Медиана: {mp_per_courier['len'].median():.1f}")
print(f"   • Максимум: {mp_per_courier['len'].max()}")
print(f"   • Минимум: {mp_per_courier['len'].min()}")

=== АНАЛИЗ КУРЬЕРОВ ===
🚚 Общее количество курьеров: 280

⏱️ Статистики сервисных времен:
   • Общее количество записей: 390040
   • Уникальных полигонов: 1393
   • Диапазон времени: 6 - 360 сек
   • Среднее время: 63.1 сек
   • Медиана времени: 15.0 сек

📊 Полигоны на курьера:
   • Среднее: 1393.0
   • Медиана: 1393.0
   • Максимум: 1393
   • Минимум: 1393


In [ ]:
db_path = "durations.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"✅ Подключились к базе: {db_path}")
print(f"📊 Размер базы: {Path(db_path).stat().st_size / (1024**3):.1f} GB")

✅ Подключились к базе: durations.sqlite
📊 Размер базы: 12.4 GB


In [ ]:
print("=== АНАЛИЗ МАТРИЦЫ РАССТОЯНИЙ ===")

cursor.execute("SELECT COUNT(*) FROM dists")
total_records = cursor.fetchone()[0]
print(f"📊 Общее количество записей: {total_records:,}")

cursor.execute("SELECT MIN(d), MAX(d), AVG(d), COUNT(*) FROM dists")
stats = cursor.fetchone()
print(f"\n📏 Статистики расстояний:")
print(f"   • Минимум: {stats[0]} сек")
print(f"   • Максимум: {stats[1]} сек") 
print(f"   • Среднее: {stats[2]:.1f} сек")
print(f"   • Общее количество: {stats[3]:,}")

cursor.execute("SELECT COUNT(*) FROM dists WHERE f = 1 OR t = 1")
warehouse_connections = cursor.fetchone()[0]
print(f"\n🏢 Связи со складом (ID=1): {warehouse_connections:,} записей")

if warehouse_connections > 0:
    cursor.execute("SELECT AVG(d) FROM dists WHERE f = 1 OR t = 1")
    warehouse_avg = cursor.fetchone()[0]
    print(f"   • Среднее расстояние до склада: {warehouse_avg:.1f} сек")

print(f"\n📈 Квантили (из выборки 100k записей):")
cursor.execute("SELECT d FROM dists ORDER BY RANDOM() LIMIT 100000")
sample_distances = [row[0] for row in cursor.fetchall()]
sample_distances.sort()

for q in [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]:
    idx = int(q * len(sample_distances))
    value = sample_distances[idx]
    print(f"   • {q*100:.0f}%: {value:.1f} сек")

=== АНАЛИЗ МАТРИЦЫ РАССТОЯНИЙ ===
📊 Общее количество записей: 406,405,440

📏 Статистики расстояний:
   • Минимум: 0 сек
   • Максимум: 17354 сек
   • Среднее: 3703.0 сек
   • Общее количество: 406,405,440


OperationalError: near "from": syntax error

In [ ]:
print("\n📊 Загружаем выборку для визуализации...")
cursor.execute("SELECT f, t, d FROM dists ORDER BY RANDOM() LIMIT 100000")
sample_data = cursor.fetchall()

distances_df = pl.DataFrame(sample_data, schema=['from', 'to', 'dist'])
print(f"✅ Загружено {len(distances_df):,} записей для анализа")

print(f"\n📊 Анализ выборки с Polars:")
print(f"   • Среднее расстояние: {distances_df['dist'].mean():.1f} сек")
print(f"   • Медиана расстояния: {distances_df['dist'].median():.1f} сек")
print(f"   • Стандартное отклонение: {distances_df['dist'].std():.1f} сек")
print(f"   • Минимум: {distances_df['dist'].min()} сек")
print(f"   • Максимум: {distances_df['dist'].max()} сек")

print(f"\n📈 Квантили выборки:")
for q in [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]:
    value = distances_df['dist'].quantile(q)
    print(f"   • {q*100}%: {value:.1f} сек")

conn.close()


📊 Загружаем выборку для визуализации...
✅ Загружено 100,000 записей для анализа

📊 Анализ выборки с Polars:
   • Среднее расстояние: 3709.8 сек
   • Медиана расстояния: 3333.5 сек
   • Стандартное отклонение: 2244.8 сек
   • Минимум: 1 сек
   • Максимум: 15465 сек

📈 Квантили выборки:
   • 10.0%: 1215.0 сек
   • 25.0%: 1946.0 сек
   • 50.0%: 3334.0 сек
   • 75.0%: 4944.0 сек
   • 90.0%: 6829.0 сек
   • 95.0%: 8241.0 сек
   • 99.0%: 10251.0 сек


/var/folders/_z/fhkbg7k116z3b681d0104yjh0000gn/T/ipykernel_44346/1668496731.py:7: DataOrientationWarning:

Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.



In [ ]:
print("Создаем визуализации...")

mp_stats_pd = mp_stats.to_pandas()
orders_pd = orders_df.to_pandas()

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=list(range(len(mp_stats_pd))), 
    y=mp_stats_pd['len'], 
    mode='lines', 
    name='Заказы по полигонам', 
    line=dict(color='blue')
))
fig1.update_layout(title="Распределение заказов по микрополигонам", 
                  xaxis_title="Полигон (по убыванию)", 
                  yaxis_title="Количество заказов")
fig1.show()

fig2 = go.Figure()
top_20 = mp_stats_pd.head(20)
fig2.add_trace(go.Bar(
    x=top_20['MpId'], 
    y=top_20['len'], 
    name='Топ-20 полигонов', 
    marker_color='lightcoral'
))
fig2.update_layout(title="Топ-20 полигонов по количеству заказов", 
                  xaxis_title="ID полигона", 
                  yaxis_title="Количество заказов")
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Histogram(
    x=mp_stats_pd['len'], 
    nbinsx=50, 
    name='Распределение', 
    marker_color='lightgreen'
))
fig3.update_layout(title="Распределение количества заказов по полигонам", 
                  xaxis_title="Количество заказов в полигоне", 
                  yaxis_title="Количество полигонов")
fig3.show()

fig4 = go.Figure()

import plotly.colors as pc
colors = pc.qualitative.Set3 + pc.qualitative.Pastel1 + pc.qualitative.Set1
color_map = {}

for mp_id in orders_pd['MpId'].unique():
    if mp_id not in color_map:
        color_map[mp_id] = colors[len(color_map) % len(colors)]
    
    mask = orders_pd['MpId'] == mp_id
    fig4.add_trace(go.Scatter(
        x=orders_pd[mask]['Long'], 
        y=orders_pd[mask]['Lat'], 
        mode='markers', 
        name=f'Полигон {mp_id}',
        marker=dict(size=3, color=color_map[mp_id], opacity=0.7),
        hovertemplate='<b>Заказ ID:</b> %{text}<br><b>Полигон:</b> %{customdata}<extra></extra>',
        text=orders_pd[mask]['ID'],
        customdata=orders_pd[mask]['MpId']
    ))

fig4.add_trace(go.Scatter(
    x=[warehouse_info['Long']], 
    y=[warehouse_info['Lat']], 
    mode='markers', 
    name='Склад', 
    marker=dict(size=20, color='black', symbol='star'),
    hovertemplate='<b>Склад ID:</b> %{text}<extra></extra>',
    text=[warehouse_info['ID']]
))

fig4.update_layout(title="Географическое распределение заказов по полигонам", 
                  xaxis_title="Долгота", 
                  yaxis_title="Широта")
fig4.show()

Создаем визуализации...


In [19]:
# Визуализация матрицы расстояний
# Конвертируем для plotly
distances_pd = distances_df.to_pandas()

# 1. Распределение расстояний
fig5 = go.Figure()
fig5.add_trace(go.Histogram(
    x=distances_pd['dist'], 
    nbinsx=100, 
    name='Распределение', 
    marker_color='lightgreen'
))
fig5.update_layout(title="Распределение расстояний", 
                  xaxis_title="Расстояние (секунды)", 
                  yaxis_title="Количество")
fig5.show()

# 2. Квантили
fig6 = go.Figure()
quantile_values = [distances_df['dist'].quantile(q) for q in [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]]
quantile_labels = ['10%', '25%', '50%', '75%', '90%', '95%', '99%']
fig6.add_trace(go.Scatter(
    x=quantile_labels, 
    y=quantile_values, 
    mode='lines+markers', 
    name='Квантили', 
    line=dict(color='red'),
    marker=dict(size=8)
))
fig6.update_layout(title="Квантили расстояний", 
                  xaxis_title="Квантиль", 
                  yaxis_title="Расстояние (секунды)")
fig6.show()

# 3. Расстояния до склада
warehouse_sample = distances_df.filter((pl.col('from') == 1) | (pl.col('to') == 1))
if len(warehouse_sample) > 0:
    fig7 = go.Figure()
    warehouse_pd = warehouse_sample.to_pandas()
    fig7.add_trace(go.Histogram(
        x=warehouse_pd['dist'], 
        nbinsx=50, 
        name='До склада', 
        marker_color='orange'
    ))
    fig7.update_layout(title="Расстояния до склада", 
                      xaxis_title="Расстояние (секунды)", 
                      yaxis_title="Количество")
    fig7.show()
else:
    print("⚠️ В выборке нет связей со складом (ID=1)")

# 4. Топ-20 самых длинных маршрутов
fig8 = go.Figure()
top_longest = distances_df.sort('dist', descending=True).head(20)
top_longest_pd = top_longest.to_pandas()
fig8.add_trace(go.Bar(
    x=[f"{row['from']}-{row['to']}" for _, row in top_longest_pd.iterrows()], 
    y=top_longest_pd['dist'], 
    name='Самые длинные', 
    marker_color='red'
))
fig8.update_layout(title="Топ-20 самых длинных маршрутов", 
                  xaxis_title="Маршрут (from-to)", 
                  yaxis_title="Расстояние (секунды)")
fig8.show()

⚠️ В выборке нет связей со складом (ID=1)


In [20]:
# Сводная статистика
print("=== СВОДНАЯ СТАТИСТИКА ===")
print(f"📦 Заказы: {len(orders_df)} (в {orders_df.select('MpId').n_unique()} микрополигонах)")
print(f"🚚 Курьеры: {len(couriers_df)}")
print(f"📏 Матрица расстояний: {total_records:,} записей в SQLite")
print(f"⏰ Время работы: 8:00-20:00 (12 часов = 43,200 секунд)")
print(f"💰 Штраф за неразмещенный заказ: 3,000 секунд")

print(f"\n📊 Статистики по микрополигонам:")
print(f"   • Среднее заказов на полигон: {mp_stats['len'].mean():.1f}")
print(f"   • Медиана заказов на полигон: {mp_stats['len'].median():.1f}")
print(f"   • Максимум заказов в полигоне: {mp_stats['len'].max()}")
print(f"   • Минимум заказов в полигоне: {mp_stats['len'].min()}")

print(f"\n⏱️ Статистики сервисных времен:")
print(f"   • Среднее время: {service_df['service_time'].mean():.1f} сек")
print(f"   • Медиана времени: {service_df['service_time'].median():.1f} сек")
print(f"   • Максимум времени: {service_df['service_time'].max()} сек")

print(f"\n📏 Статистики расстояний:")
print(f"   • Среднее расстояние: {stats[2]:.1f} сек")
print(f"   • Максимум расстояния: {stats[1]} сек")
print(f"   • {warehouse_connections:,} связей со складом")

if warehouse_connections > 0:
    print(f"   • Среднее до склада: {warehouse_avg:.1f} сек")

=== СВОДНАЯ СТАТИСТИКА ===
📦 Заказы: 20160 (в 1394 микрополигонах)
🚚 Курьеры: 280
📏 Матрица расстояний: 406,405,440 записей в SQLite
⏰ Время работы: 8:00-20:00 (12 часов = 43,200 секунд)
💰 Штраф за неразмещенный заказ: 3,000 секунд

📊 Статистики по микрополигонам:
   • Среднее заказов на полигон: 14.5
   • Медиана заказов на полигон: 15.0
   • Максимум заказов в полигоне: 15
   • Минимум заказов в полигоне: 1

⏱️ Статистики сервисных времен:
   • Среднее время: 63.1 сек
   • Медиана времени: 15.0 сек
   • Максимум времени: 360 сек

📏 Статистики расстояний:
   • Среднее расстояние: 3703.0 сек
   • Максимум расстояния: 17354 сек
   • 0 связей со складом
